In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from cmdstanpy import CmdStanModel
import os
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

In [ ]:
# Set random seed
np.random.seed(42)

# Load preprocessed data
train_data = pd.read_csv('processed_train_data.csv')
test_data = pd.read_csv('processed_test_data.csv')

# Check if data is empty
if len(train_data) == 0:
    raise ValueError("Train data is empty. Check preprocessing step.")
if len(test_data) == 0:
    print("Warning: Test data is empty. Predictions will be skipped.")

In [ ]:
## Define features (exclude quality flags and season)
categorical_cols = ['is_raining', 'day_of_week', 'month']
# Corrected column name based on actual data
numerical_cols = ['max_temp',
                  'precipitation', 'pressure',
                  'humidity', 'cloud_cover']
features = numerical_cols + categorical_cols
target = 'passenger_count'

In [ ]:
# Prepare training and test sets
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features] if len(test_data) > 0 else pd.DataFrame(columns=X_train.columns)
y_test = test_data[target] if len(test_data) > 0 else pd.Series()

In [ ]:
# Preprocess categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),
        ('num', 'passthrough', numerical_cols)
    ])
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test) if len(X_test) > 0 else np.array([])

# Get feature names
cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
feature_names = list(cat_feature_names) + numerical_cols
dow_indices = [i for i, name in enumerate(feature_names) if name.startswith('day_of_week_')]

# Prior

In [ ]:
# Update stan_data to include the number of day_of_week indices
stan_data = {
    'N': len(X_train_transformed),
    'K': X_train_transformed.shape[1],
    'X': X_train_transformed,
    'num_dow_indices': len(dow_indices),  # Add the size of the dow_indices array
    'dow_indices': [i+1 for i, name in enumerate(feature_names) if name.startswith('day_of_week_')]  # 1-based indexing for Stan
}

In [ ]:
# Update the Stan code to declare dow_indices with the correct size
priors_stan = """
data {
  int<lower=0> N;          // Number of samples to generate
  int<lower=0> K;          // Number of features
  matrix[N, K] X;          // Feature matrix
  int<lower=0> num_dow_indices; // Number of day_of_week features
  array[num_dow_indices] int<lower=1, upper=K> dow_indices;  // Indices of day_of_week features
}
parameters {
  real beta0;              // Intercept
  vector[K] beta;          // Feature coefficients
  real mu_dow;             // Mean of day_of_week coefficients
  real<lower=0> sigma_dow; // SD of day_of_week coefficients
  real<lower=0> sigma;     // Noise standard deviation
}
model {
  // Priors
  beta0 ~ normal(0, 0.9);
  for (k in 1:K) {
    // Check if the current index k is one of the dow_indices
    int is_dow = 0; // Flag to indicate if k is a dow index
    for (i in 1:size(dow_indices)) { // Iterate through dow_indices
      if (k == dow_indices[i]) {
        is_dow = 1; // Set flag if match found
      }
    }

    if (is_dow == 1) { // Use the flag in the conditional
      beta[k] ~ normal(mu_dow, sigma_dow);
    } else {
      beta[k] ~ normal(0, 1);
    }
  }
  mu_dow ~ normal(0, 0.3);
  sigma_dow ~ normal(0, 0.3);
  sigma ~ normal(0, 0.3);
}
generated quantities {
  vector[N] y;             // Generated target variable
  array[K] vector[N] y_per_feature;  // Per-feature prior predictive
  for (n in 1:N) {
    y[n] = normal_rng(X[n] * beta + beta0, sigma);
    for (k in 1:K) {
      y_per_feature[k, n] = normal_rng(X[n, k] * beta[k] + beta0, sigma);
    }
  }
}
"""
# Save Stan model
with open('hierarchical_regression_priors.stan', 'w') as f:
    f.write(priors_stan)

In [ ]:
model = CmdStanModel(stan_file='hierarchical_regression_priors.stan')
fit = model.sample(data=stan_data, chains=4, iter_sampling=2000, iter_warmup=500, adapt_delta=0.9 ,seed=42)

In [ ]:
# Extract prior predictive samples
y_prior = fit.stan_variable('y').flatten()  # Shape: (n_samples * N,)
y_per_feature = fit.stan_variable('y_per_feature')  # Shape: (n_samples, K, N)
real_data = y_train.values

# Visualization: Histograms for real and prior predictive passenger_count for each feature
n_features = len(feature_names)
fig, axes = plt.subplots(nrows=(n_features + 2) // 3, ncols=min(n_features, 3), figsize=(15, 5 * ((n_features + 2) // 3)))
axes = axes.flatten() if n_features > 1 else [axes]
bin_range = (real_data.min() * 1.5, real_data.max() * 1.5)
# Plot for each feature
for idx, feature in enumerate(feature_names):
    # Extract prior predictive samples for this feature (flatten across samples and observations)
    y_feature = y_per_feature[:, idx, :].flatten()  # Shape: (n_samples * N,)

    # Plot histograms with plt.hist
    axes[idx].hist(real_data, bins=20, density=True, alpha=0.5, label='Real passenger_count', color='blue',
                   range=bin_range, histtype='stepfilled')
    axes[idx].hist(y_feature, bins=20, density=True, alpha=0.5, label=f'Prior Predictive ({feature})', color='orange',
                   range=bin_range, histtype='stepfilled')

    axes[idx].set_title(f'Effect of {feature} on passenger_count')
    axes[idx].set_xlabel('passenger_count')
    axes[idx].set_ylabel('Density')
    axes[idx].legend()

# Adjust layout and display
plt.tight_layout()
plt.show()

# Posterior

In [ ]:
# Define data
stan_data = {
    'N': len(X_train_transformed),
    'K': X_train_transformed.shape[1],
    'X': X_train_transformed,
    'y': y_train.values,
    'dow_indices': [i+1 for i, name in enumerate(feature_names) if name.startswith('day_of_week_')],
    'num_dow_indices': len([i+1 for i, name in enumerate(feature_names) if name.startswith('day_of_week_')]) # Add num_dow_indices here
}

In [ ]:
# Define Stan model with log-likelihood for arviz
stan_code = """
data {
  int<lower=0> N;          // Number of training samples
  int<lower=0> K;          // Number of features
  matrix[N, K] X;          // Feature matrix
  vector[N] y;             // Target variable
  int<lower=0> num_dow_indices; // Number of day_of_week features (Added for loop)
  array[num_dow_indices] int<lower=1, upper=K> dow_indices;  // Indices of day_of_week features
}
parameters {
  real beta0;              // Intercept
  vector[K] beta;          // Feature coefficients
  real mu_dow;             // Mean of day_of_week coefficients
  real<lower=0> sigma_dow; // SD of day_of_week coefficients
  real<lower=0> sigma;     // Noise standard deviation
}
model {
  vector[N] mu;
  // Priors
  beta0 ~ normal(0, 0.9);
  for (k in 1:K) {
    // Check if the current index k is one of the dow_indices
    int is_dow = 0; // Flag to indicate if k is a dow index
    for (i in 1:num_dow_indices) { // Iterate through dow_indices
      if (k == dow_indices[i]) {
        is_dow = 1; // Set flag if match found
        break; // Exit the inner loop once a match is found
      }
    }

    if (is_dow == 1) {
      beta[k] ~ normal(mu_dow, sigma_dow);
    } else {
      beta[k] ~ normal(0, 1);
    }
  }
  mu_dow ~ normal(0, 0.3);
  sigma_dow ~ normal(0, 0.3);
  sigma ~ normal(0, 0.3);

  // Likelihood
  for (n in 1:N) {
    mu[n] = beta0 + dot_product(X[n], beta);
  }
  y ~ normal(mu, sigma);
}
generated quantities {
  vector[N] y_pred;        // Posterior predictive checks
  vector[N] log_lik;       // Log likelihood for model evaluation
  array[K] vector[N] y_per_feature;  // Per-feature posterior predictive
  for (n in 1:N) {
    y_pred[n] = normal_rng(X[n] * beta + beta0, sigma);
    log_lik[n] = normal_lpdf(y[n] | X[n] * beta + beta0, sigma);
    for (k in 1:K) {
      y_per_feature[k, n] = normal_rng(X[n, k] * beta[k] + beta0, sigma);
    }
  }
}
"""

# Save Stan model
with open('hierarchical_regression_poterior.stan', 'w') as f:
    f.write(stan_code)

In [ ]:
# Compile and fit model
model = CmdStanModel(stan_file='hierarchical_regression_poterior.stan')
posterior_fit = model.sample(data=stan_data, chains=4, iter_sampling=600, iter_warmup=300, adapt_delta=0.90 ,seed=42)

In [ ]:
# Convert to arviz InferenceData for posterior analysis
idata = az.from_cmdstanpy(posterior=posterior_fit, log_likelihood='log_lik')

# Extract posterior predictive samples
y_pred = fit.stan_variable('y').mean(axis=0)  # Mean posterior predictive
y_per_feature = fit.stan_variable('y_per_feature')  # Posterior predictive per feature

# Visualization: Histograms for real data vs posterior predictive for each feature
n_features = len(feature_names)
fig, axes = plt.subplots(nrows=(n_features + 2) // 3, ncols=min(n_features, 3), figsize=(15, 5 * ((n_features + 2) // 3)))
axes = axes.flatten() if n_features > 1 else [axes]
bin_range = (real_data.min() * 1.5, real_data.max() * 1.5)
real_data = y_train.values
for idx, feature in enumerate(feature_names):
    y_feature = y_per_feature[:, idx, :].mean(axis=0)  # Mean posterior predictive per feature
    axes[idx].hist(real_data, bins=50, density=True, alpha=0.5, label='Real passenger_count', color='blue', range=bin_range, histtype='stepfilled')
    axes[idx].hist(y_feature, bins=50, density=True, alpha=0.5, label=f'Posterior Predictive ({feature})', color='green', range=bin_range, histtype='stepfilled')
    axes[idx].set_title(f'Effect of {feature} on passenger_count')
    axes[idx].set_xlabel('passenger_count')
    axes[idx].set_ylabel('Density')
    axes[idx].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Check sampling diagnostics
summary = fit.summary()
rhat = summary['R_hat'].max() if 'R_hat' in summary.columns else float('nan')
# Try 'n_eff' as the column name, fallback to None if missing
n_eff = summary['n_eff'].min() if 'n_eff' in summary.columns else None
print(f"Sampling Diagnostics: Max R-hat = {rhat:.4f}, Min N_Eff = {n_eff if n_eff is not None else 'N/A'}")
if rhat > 1.1 or (n_eff is not None and n_eff < 100):
    print("Warning: Sampling issues detected. Consider increasing iter_sampling or adapt_delta, or check model specification.")

In [ ]:
# Posterior predictive analysis
if len(X_test_transformed) > 0:
    y_pred = fit.stan_variable('y_pred').mean(axis=0)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Posterior Predictive Metrics (Model 2):")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")
    # Check consistency
    y_pred_samples = fit.stan_variable('y_pred')
    credible_intervals = np.percentile(y_pred_samples, [2.5, 97.5], axis=0)
    within_ci = np.mean((y_test >= credible_intervals[0]) & (y_test <= credible_intervals[1]))
    print(f"Proportion of test data within 95% credible intervals: {within_ci:.2f}")
    if within_ci < 0.9:
        print("Warning: Less than 90% of test data within credible intervals. Consider non-linear effects or additional features.")

In [ ]:
# Visualization: Actual vs. Predicted Passenger Counts
    if 'date' in test_data.columns:
        plt.figure(figsize=(12, 6))
        plt.plot(test_data['date'], y_test, label='Actual', color='blue')
        plt.plot(test_data['date'], y_pred, label='Predicted', color='red', alpha=0.7)
        plt.title('Actual vs. Predicted Passenger Counts (Model 2)')
        plt.xlabel('Date')
        plt.ylabel('Normalized Passenger Count')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('actual_vs_predicted_model2.png')
        plt.close()
    else:
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', alpha=0.7)
        plt.title('Actual vs. Predicted Passenger Counts (Model 2)')
        plt.xlabel('Index')
        plt.ylabel('Normalized Passenger Count')
        plt.legend()
        plt.tight_layout()
        plt.savefig('actual_vs_predicted_model2.png')
        plt.close()

    # Visualization: Residuals Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(12, 6))
    plt.scatter(test_data['date'] if 'date' in test_data.columns else y_test.index, residuals, color='purple', alpha=0.5)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    plt.title('Residuals of Predicted vs. Actual Passenger Counts (Model 2)')
    plt.xlabel('Date' if 'date' in test_data.columns else 'Index')
    plt.ylabel('Residuals')
    plt.tight_layout()
    plt.savefig('residuals_model2.png')
    plt.close()

In [ ]:
# Parameter marginal distributions
beta_samples = fit.stan_variable('beta')
beta0_samples = fit.stan_variable('beta0')
sigma_samples = fit.stan_variable('sigma')
mu_dow_samples = fit.stan_variable('mu_dow')
sigma_dow_samples = fit.stan_variable('sigma_dow')
print("\nParameter Summaries (Model 2):")
for i, name in enumerate(feature_names):
    mean, std = beta_samples[:, i].mean(), beta_samples[:, i].std()
    ci = np.percentile(beta_samples[:, i], [2.5, 97.5])
    print(f"{name}: Mean = {mean:.4f}, SD = {std:.4f}, 95% CI = [{ci[0]:.4f}, {ci[1]:.4f}]")
print(f"beta0: Mean = {beta0_samples.mean():.4f}, SD = {beta0_samples.std():.4f}, 95% CI = [{np.percentile(beta0_samples, 2.5):.4f}, {np.percentile(beta0_samples, 97.5):.4f}]")
print(f"mu_dow: Mean = {mu_dow_samples.mean():.4f}, SD = {mu_dow_samples.std():.4f}, 95% CI = [{np.percentile(mu_dow_samples, 2.5):.4f}, {np.percentile(mu_dow_samples, 97.5):.4f}]")
print(f"sigma_dow: Mean = {sigma_dow_samples.mean():.4f}, SD = {sigma_dow_samples.std():.4f}, 95% CI = [{np.percentile(sigma_dow_samples, 2.5):.4f}, {np.percentile(sigma_dow_samples, 97.5):.4f}]")
print(f"sigma: Mean = {sigma_samples.mean():.4f}, SD = {sigma_samples.std():.4f}, 95% CI = [{np.percentile(sigma_samples, 2.5):.4f}, {np.percentile(sigma_samples, 97.5):.4f}]")

In [ ]:
# Plot parameter histograms
plt.figure(figsize=(12, 8))
for i, name in enumerate(feature_names[:4]):
    plt.subplot(2, 2, i+1)
    plt.hist(beta_samples[:, i], bins=30, density=True)
    plt.title(f'Posterior: {name} (Model 2)')
    plt.xlabel('Value')
    plt.ylabel('Density')
plt.tight_layout()
plt.savefig('parameter_histograms_model2.png')
plt.show()

In [ ]:
# Save sampling output to CSV files
csv_dir = "csv_output_model2"
fit.save_csvfiles(dir=csv_dir)

In [ ]:
# Compute information criteria with arviz using from_cmdstanpy
idata = az.from_cmdstanpy(posterior=fit, log_likelihood='log_lik')
waic = az.waic(idata)
loo = az.loo(idata)
print(f"\nInformation Criteria (Model 2):")
print(f"WAIC: {waic.elpd_waic} (+/- {waic.se})")
print(f"PSIS-LOO: {loo.elpd_loo} (+/- {loo.se})")
if any(loo.pareto_k > 0.7):
    print("Warning: High Pareto k values detected. Results may be unreliable.")